In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import random
import re
import spacy

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim
from gensim import corpora, models, similarities, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
# logging (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
df = pd.read_csv('new_negative.csv')
df

/home/ipsita/anaconda3/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


,Date,Stars,Comment,Target,Year,Month,totalwords
0,2019-10-29,1,I had an accident with an Uber driver in Mexic...,0,2019,10,153
1,2019-10-28,1,I have had my account completely hacked to whe...,0,2019,10,155
2,2019-10-27,1,I requested an 8 mile ride in Boston on a Satu...,0,2019,10,45
3,2019-10-27,1,I've been driving off and on with the company ...,0,2019,10,120
4,2019-10-25,1,Uber is overcharging for Toll fees. When In Fl...,0,2019,10,109
...,...,...,...,...,...,...,...
1239,2017-01-08,1,Why is it if they take money out of your accou...,0,2017,1,195
1240,2017-01-05,1,I started driving with Uber late October 2016....,0,2017,1,70
1241,2017-01-04,1,Uber drivers are cheater. I must say that my s...,0,2017,1,66
1242,2017-01-04,1,We used Uber to travel 8.2 miles from our hote...,0,2017,1,139


In [2]:
nlp = spacy.load('en_core_web_sm')
docs = list(nlp.pipe(df.Comment))
df['spacy_doc'] = docs

In [3]:
docs_clean = [[w.lemma_.lower() for w in doc if (not w.is_stop and not w.is_punct and not w.like_num) or (w.lemma_=='not')] for doc in docs]
df['docs_clean'] = docs_clean

In [4]:
df.head()

,Date,Stars,Comment,Target,Year,Month,totalwords,spacy_doc,docs_clean
0,2019-10-29,1,I had an accident with an Uber driver in Mexic...,0,2019,10,153,"(I, had, an, accident, with, an, Uber, driver,...","[accident, uber, driver, mexico, city, car, ge..."
1,2019-10-28,1,I have had my account completely hacked to whe...,0,2019,10,155,"(I, have, had, my, account, completely, hacked...","[account, completely, hack, not, sign, view, s..."
2,2019-10-27,1,I requested an 8 mile ride in Boston on a Satu...,0,2019,10,45,"(I, requested, an, 8, mile, ride, in, Boston, ...","[request, mile, ride, boston, saturday, night,..."
3,2019-10-27,1,I've been driving off and on with the company ...,0,2019,10,120,"(I, 've, been, driving, off, and, on, with, th...","[drive, company, drive, new, place, notice, ub..."
4,2019-10-25,1,Uber is overcharging for Toll fees. When In Fl...,0,2019,10,109,"(Uber, is, overcharging, for, Toll, fees, ., W...","[uber, overcharge, toll, fee, florida, tolls, ..."


In [5]:
docs_list_clean = [' '.join(doc) for doc in docs_clean]
docs_list_clean[:-3]

['accident uber driver mexico city car get mirror brakes not work properly get accident twice driver conversation unpleasant foreigner curious ask bring mexico reply tourist conversation rude ask come look mexican husband answer keep quiet take wrong route request stop have mind uberx. continue ask sleep literally stop car ask wait text ask let different driver lock door allow',
 'account completely hack not sign view spend $ 1k uber ride credit debit card link account dial number available safety line tell need wait department email hear hour prompt message not help day receive phone helpful support email nightmare bad customer service experience go file police report person driver catch fraudulently account look attorney uber not make identity theft priority rectify situation end',
 'request mile ride boston saturday night 1:00 come $ app lyft rate end take regular cab cost $ beware high rate',
 'drive company drive new place notice uber take fare passenger pay $ uber take $ give $ d

In [6]:
letters_only = [re.sub("[^a-zA-Z]", " ", w)for w in docs_list_clean]
letters_only

['accident uber driver mexico city car get mirror brakes not work properly get accident twice driver conversation unpleasant foreigner curious ask bring mexico reply tourist conversation rude ask come look mexican husband answer keep quiet take wrong route request stop have mind uberx  continue ask sleep literally stop car ask wait text ask let different driver lock door allow',
 'account completely hack not sign view spend    k uber ride credit debit card link account dial number available safety line tell need wait department email hear hour prompt message not help day receive phone helpful support email nightmare bad customer service experience go file police report person driver catch fraudulently account look attorney uber not make identity theft priority rectify situation end',
 'request mile ride boston saturday night      come   app lyft rate end take regular cab cost   beware high rate',
 'drive company drive new place notice uber take fare passenger pay   uber take   give   d

In [7]:
vec = TfidfVectorizer(stop_words = 'english', max_df = 0.5, max_features = 1000)
X = vec.fit_transform(letters_only)
df_X = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df_X.head()

,able,absolute,absolutely,accept,access,accident,accord,account,acknowledge,act,...,worth,write,wrong,xl,year,yell,yellow,yes,yesterday,york
0,0.0,0.0,0.0,0.0,0.0,0.322726,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.111899,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.263864,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.095039,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
len(vec.get_feature_names())

1000

In [9]:
num_topics = 4
#topics = TruncatedSVD(num_topics)
topics = NMF(num_topics)
doc_topic = topics.fit_transform(X)

/home/ipsita/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/home/ipsita/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [10]:
topic_word = pd.DataFrame(topics.components_.round(3),
             columns = vec.get_feature_names()).T.add_prefix('topic_')
topic_word

,topic_0,topic_1,topic_2,topic_3
able,0.103,0.054,0.000,0.060
absolute,0.035,0.010,0.000,0.000
absolutely,0.065,0.003,0.030,0.010
accept,0.013,0.077,0.076,0.040
access,0.096,0.016,0.000,0.000
...,...,...,...,...
yell,0.000,0.000,0.002,0.024
yellow,0.004,0.000,0.013,0.018
yes,0.000,0.000,0.030,0.026
yesterday,0.032,0.019,0.017,0.032


In [11]:
doc_topic_df = pd.DataFrame(doc_topic).add_prefix('topic_').round(3)

doc_topic_df[['Comment']] = df[['Comment']]
doc_topic_df[['Target']] = df[['Target']]
doc_topic_df[['Year']] = df[['Year']]
doc_topic_df[['Month']] = df[['Month']]
doc_topic_df.head()

,topic_0,topic_1,topic_2,topic_3,Comment,Target,Year,Month
0,0.032,0.000,0.010,0.078,I had an accident with an Uber driver in Mexic...,0,2019,10
1,0.134,0.094,0.000,0.000,I have had my account completely hacked to whe...,0,2019,10
2,0.004,0.000,0.082,0.036,I requested an 8 mile ride in Boston on a Satu...,0,2019,10
3,0.042,0.000,0.064,0.040,I've been driving off and on with the company ...,0,2019,10
4,0.035,0.010,0.172,0.000,Uber is overcharging for Toll fees. When In Fl...,0,2019,10


In [12]:
doc_topic_df.sort_values(by='topic_2', ascending=False).head(10)

,topic_0,topic_1,topic_2,topic_3,Comment,Target,Year,Month
92,0.000,0.021,0.206,0.011,I have been charged several time for wait time...,0,2019,5
1093,0.000,0.000,0.205,0.000,They charging unfair money for the trips! I ha...,0,2017,4
386,0.021,0.026,0.196,0.000,"I urge you, do not use Uber. They are well kno...",0,2018,10
355,0.000,0.000,0.191,0.000,Uber charged me an additional $42 for a ride t...,0,2018,11
994,0.000,0.000,0.188,0.000,2 times I had the incorrect fare charged. On 2...,0,2017,6
997,0.000,0.012,0.179,0.055,Uber mentions standard upfront charges for the...,0,2017,6
315,0.000,0.000,0.176,0.000,They charged me a $125 transaction fee for a t...,0,2018,12
138,0.000,0.038,0.175,0.020,In my most recent requests for an Uber Ride he...,0,2019,4
553,0.002,0.000,0.173,0.014,"Not only was my driver unable to find me, we h...",0,2018,6
4,0.035,0.010,0.172,0.000,Uber is overcharging for Toll fees. When In Fl...,0,2019,10


In [13]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [14]:
display_topics(topics, vec.get_feature_names(), 10)


Topic  0
phone, customer, account, service, email, number, app, try, support, company

Topic  1
card, credit, gift, account, use, purchase, charge, payment, ride, refund

Topic  2
charge, ride, trip, fare, fee, price, airport, time, pay, request

Topic  3
cancel, minute, order, wait, pick, say, time, car, min, come


In [36]:
doc_topic_df.groupby('Target', as_index=False)['topic_0'].mean()

,Target,topic_0
0,0,0.040891


In [37]:
doc_topic_df.groupby('Target', as_index=False)['topic_1'].mean()

,Target,topic_1
0,0,0.026349


In [38]:
doc_topic_df.groupby('Target', as_index=False)['topic_2'].mean()

,Target,topic_2
0,0,0.044862


In [39]:
doc_topic_df.groupby('Target', as_index=False)['topic_3'].mean()

,Target,topic_3
0,0,0.047393


In [15]:
for i in doc_topic_df.sort_values(by='topic_0', ascending=False).head(10)['Comment'].values:
    print(i)
    print()

Today I left my cell phone in the Uber. Every avenue I have taken to try to contact Uber has led to a dead end. I need the verification code sent to my phone in order to login from my desktop. But I do not have my phone. So I cannot get the verification code. There is a page purportedly just for resolving this issue. But it no longer works. I emailed customer service for help getting into my account and at 12 hours later, I have not had a response. I emailed them all day, actually. I called the only number available. That number is for drivers. There are no telephone numbers for riders unless you have the app on your phone. But I don't have my phone. The lady on the driver line promised to help me, then she put me on hold for 40 minutes. So I finally hung up. While I was on the driver line, I heard all the things Uber is doing for their drivers, as a "driver centric company." They do not care about the consumer at all. This is wrong. Just plain wrong.

I lost my cell phone in an Uber v

In [30]:
vec1 = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.9, 
                                min_df = 6)
X_LDA = vec1.fit_transform(letters_only)
print(X_LDA.shape)

(1244, 1066)


In [31]:
corpus = matutils.Sparse2Corpus(X_LDA.T)

In [32]:
id2word = dict((v, k) for k, v in vec1.vocabulary_.items())

In [33]:
len(id2word)

1066

In [34]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=4, id2word=id2word, passes=5)

2021-11-11 22:12:04,216 : INFO : using symmetric alpha at 0.25
2021-11-11 22:12:04,222 : INFO : using symmetric eta at 0.25
2021-11-11 22:12:04,226 : INFO : using serial LDA version on this node
2021-11-11 22:12:04,234 : INFO : running online (multi-pass) LDA training, 4 topics, 5 passes over the supplied corpus of 1244 documents, updating model once every 1244 documents, evaluating perplexity every 1244 documents, iterating 50x with a convergence threshold of 0.001000
2021-11-11 22:12:04,235 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2021-11-11 22:12:06,393 : INFO : -7.578 per-word bound, 191.1 perplexity estimate based on a held-out corpus of 1244 documents with 45438 words
2021-11-11 22:12:06,400 : INFO : PROGRESS: pass 0, at document #1244/1244
2021-11-11 22:12:08,961 : INFO : topic #0 (0.250): 0.063*"uber" + 0.022*"charge" + 0.017*"time" + 0.015*"customer" + 0.014*"trip" + 0.013*"driver" + 0.

In [35]:
lda.print_topics()

2021-11-11 22:12:22,279 : INFO : topic #0 (0.250): 0.059*"uber" + 0.024*"charge" + 0.018*"time" + 0.018*"order" + 0.016*"trip" + 0.013*"customer" + 0.012*"driver" + 0.011*"pay" + 0.011*"service" + 0.011*"price"
2021-11-11 22:12:22,291 : INFO : topic #1 (0.250): 0.054*"uber" + 0.045*"driver" + 0.024*"ride" + 0.022*"cancel" + 0.022*"charge" + 0.016*"minute" + 0.016*"time" + 0.015*"say" + 0.012*"service" + 0.011*"customer"
2021-11-11 22:12:22,294 : INFO : topic #2 (0.250): 0.064*"uber" + 0.029*"account" + 0.022*"card" + 0.018*"ride" + 0.018*"charge" + 0.018*"email" + 0.017*"time" + 0.015*"phone" + 0.013*"credit" + 0.013*"app"
2021-11-11 22:12:22,298 : INFO : topic #3 (0.250): 0.075*"uber" + 0.055*"driver" + 0.021*"ride" + 0.017*"car" + 0.016*"drive" + 0.013*"time" + 0.011*"charge" + 0.011*"pick" + 0.008*"service" + 0.007*"customer"


[(0,
  '0.059*"uber" + 0.024*"charge" + 0.018*"time" + 0.018*"order" + 0.016*"trip" + 0.013*"customer" + 0.012*"driver" + 0.011*"pay" + 0.011*"service" + 0.011*"price"'),
 (1,
  '0.054*"uber" + 0.045*"driver" + 0.024*"ride" + 0.022*"cancel" + 0.022*"charge" + 0.016*"minute" + 0.016*"time" + 0.015*"say" + 0.012*"service" + 0.011*"customer"'),
 (2,
  '0.064*"uber" + 0.029*"account" + 0.022*"card" + 0.018*"ride" + 0.018*"charge" + 0.018*"email" + 0.017*"time" + 0.015*"phone" + 0.013*"credit" + 0.013*"app"'),
 (3,
  '0.075*"uber" + 0.055*"driver" + 0.021*"ride" + 0.017*"car" + 0.016*"drive" + 0.013*"time" + 0.011*"charge" + 0.011*"pick" + 0.008*"service" + 0.007*"customer"')]